The dataset used in this exercise comes from the UCI ML Repository and contains data calculated from digitized images of breast masses.  

They describe the characteristics of the cell nuclei present in each image (radius, perimeter, texture, etc.). The first column gives the diagnostic result of each cell mass: 'B' for benign, 'M' for malignant.  

The objective of the exercise is to build a model in the form of a decision tree, to predict whether a cell mass is benign or malignant, based on the characteristics calculated from the biopsy image.  

In [8]:
import os

url = 'https://archive.ics.uci.edu/static/public/17/breast+cancer+wisconsin+diagnostic.zip'

import requests
import zipfile
import io
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
import pandas as pd


if not os.path.exists('wdbc.data'):
    response = requests.get(url)
    zip_file = zipfile.ZipFile(io.BytesIO(response.content))
    zip_file.extractall()




In [16]:
from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
breast_cancer_wisconsin_diagnostic = fetch_ucirepo(id=17) 
  
# data (as pandas dataframes) 
X = breast_cancer_wisconsin_diagnostic.data.features 
y = breast_cancer_wisconsin_diagnostic.data.targets 
  


df = pd.concat([X, y], axis=1)

In [22]:
X.head()

,radius1,texture1,perimeter1,area1,smoothness1,compactness1,concavity1,concave_points1,symmetry1,fractal_dimension1,...,radius3,texture3,perimeter3,area3,smoothness3,compactness3,concavity3,concave_points3,symmetry3,fractal_dimension3
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [23]:
y.head()

,Diagnosis
0,M
1,M
2,M
3,M
4,M


Decision trees (Decision Tree) are a non-parametric supervised learning method used for classification and regression. The goal is to create a model that predicts the value of a target variable by learning simple decision rules derived from the data features.

A decision tree can be described as a visual representation of a data classification algorithm following different criteria called decisions (or nodes). Each node corresponds to a test on a training variable, and each of the subsequent branches represents a result of this test. Each leaf of the tree (terminal nodes) contains a value of the target variable, a label in the case of classification.

During model training, nodes are created from "optimal" tests with respect to the training set, and this process ends when the tree leaves are homogeneous or meet a certain stopping criterion.

This decision tree thus allows, after training on a data set, to easily make predictions in the form of successive logical classification rules. The results are easily interpretable and therefore exploitable, making communication about modeling easier. It is therefore a highly appreciated and used classifier in industry.

The construction of a decision tree is generally done in 2 phases:

The first phase consists of constructing the nodes:

Starting from a training set, a recursive process of dividing the data space into increasingly pure subsamples in terms of classes is initiated, based on a predefined criterion.
The classification problem is thus broken down into a sequence of (nested) tests on a variable of the form "X>=threshold".
At each node, the best test is selected according to a certain criterion (often based on information theory, and notably on the notion of entropy), whose objective is to minimize the mixing of classes within each subset created by the different test alternatives.
This results in a succession of classification rules in the form of a tree, each end (or "leaf") indicating membership in a class.
The class allocated to a leaf is determined by the class mostly represented among the data in the training set that "fall" into that leaf.
The objective of this phase is to generate a hierarchical sequence of tests, as short as possible, which successively divides the entire training data set into disjoint subsets, such that subgroups of cases belonging to the same class are quickly detected.

The second phase is pruning:

It consists of removing poorly representative branches to maintain good predictive performance. This step requires the creation of a criterion/metric to designate branches to prune, which will depend on the algorithm used.
After pruning, the branches are replaced by terminal nodes, labeled based on the distribution of the training data (majority class).
Generally, pruning is done from the bottom up of the tree ("bottom-up"). It is based on an estimate (cross-validation, new sample, statistical estimation, etc.) of the classification error rate: a tree is pruned at a certain node if the estimated error rate at this node (by allocating the majority class) is lower than the error rate obtained by considering the terminal sub-trees.
Pruning proceeds successively (from the beginning of the branches) until all remaining sub-trees satisfy the condition on the classification error rates.






The DecisionTreeClassifier function allows you to create a classifier based on a decision tree. Many parameters can be given to it such as the criterion for evaluating partitions criterion, the maximum depth of the tree, the number of features to consider at each node, etc.  

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

dt_clf = DecisionTreeClassifier(criterion ='entropy', max_depth=4, random_state=123)
dt_clf.fit(X_train, y_train)

y_pred = dt_clf.predict(X_test)

pd.crosstab(y_test['Diagnosis'], y_pred, rownames=['Classe réelle'], colnames=['Classe prédite'])

Classe prédite,B,M
Classe réelle,,
B,72,1
M,3,38


In [25]:
feats = {}
for feature, importance in zip(X.columns, dt_clf.feature_importances_):
    feats[feature] = importance 
    
importances = pd.DataFrame.from_dict(feats, orient='index').rename(columns={0: 'Importance'})

# Displaying the 8 most important variables
importances.sort_values(by='Importance', ascending=False).head(8)

,Importance
radius3,0.620770
concave_points3,0.177674
concavity1,0.060736
texture1,0.051408
texture3,0.041369
radius2,0.020865
area3,0.016107
compactness2,0.011072


Two impurity measures can be used for decision trees in scikit-learn: Information Gain or Entropy, and the Gini index  



In summary, entropy is 0 if all samples in a node belong to the same class, and entropy is maximal if we have a uniform class distribution (i.e., when all classes in the node have equal probability).

In [27]:
dt_clf_gini = DecisionTreeClassifier(criterion='gini', max_depth=4, random_state=321)
dt_clf_gini.fit(X_train, y_train)
y_pred = dt_clf_gini.predict(X_test)
pd.crosstab(y_test['Diagnosis'], y_pred, rownames=['Classe réelle'], colnames=['Classe prédite'])


Classe prédite,B,M
Classe réelle,,
B,72,1
M,4,37


In [29]:
feats = {}
for feature, importance in zip(X.columns, dt_clf_gini.feature_importances_):
    feats[feature] = importance 

importances = pd.DataFrame.from_dict(feats, orient='index').rename(columns={0: 'Gini-importance'})

# Displaying the 8 most important variables
importances.sort_values(by='Gini-importance', ascending=False).head(8)

,Gini-importance
radius3,0.718582
concave_points3,0.112821
texture3,0.048798
concavity1,0.033920
texture1,0.030343
area3,0.016458
smoothness1,0.011231
texture2,0.009682


In conclusion, in this notebook we have seen:
Decision Tree: Visual representation of a classification algorithm, where each node corresponds to a test on a learning variable and each leaf contains a value of the target variable.

Tree Construction:

Phase 1: Creation of nodes by recursively dividing the data space into homogeneous subsets, choosing the best tests to minimize class mixing.
Phase 2: Pruning of unrepresentative branches to improve predictive performance, based on pruning criteria dependent on the algorithm used.
Impurity Measures: Two measures used are entropy and the Gini index, where entropy is minimal when samples in a node are homogeneous and maximal when the class distribution is uniform.

Tree Construction Process: The root is the first node of the tree, containing the frequency distribution of the variable to predict. The segmentation variable, here radius_worst, is used to create child nodes until pure classes are obtained or the maximum number of allowed nodes is reached.

Simple yet powerful, the advantage is that it requires no model training to make predictions, whereas this is generally the most complicated part of a Machine Learning algorithm. Decision tree-based algorithms allow for simple and quick classification problem solutions. They make no assumptions about the data and are not affected by variable measurement scale issues. They can handle both numerical and categorical variables and are easily interpretable.

In the next notebook, we will explore the Bagging & Boosting algorithms of Scikit-Learn !